## Build a Wflow Sediment model

This notebook demonstrates how to prepare a **Wflow Sediment** model using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface

Lets first check if the Wflow sediment model is recognized by HydroMT

In [1]:
# this should return "wflow, wflow_sediment"
# as well as hydromt generic models "grid_model, lumped_model, network_model (mesh_model)"
!hydromt --models

model plugins:
 - wflow (hydromt_wflow 0.8.0)
 - wflow_sediment (hydromt_wflow 0.8.0)
generic models (hydromt 0.10.1):
 - grid_model
 - vector_model
 - network_model



Using the **HydroMT build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [2]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT

  Build models from scratch.

  Example usage: --------------

  To build a wflow model for a subbasin using a point coordinates snapped to
  cells with upstream area >= 50 km2 hydromt build wflow /path/to/model_root
  -i /path/to/wflow_config.yml  -r "{'subbasin': [-7.24, 62.09], 'uparea':
  50}" -d deltares_data -d /path/to/data_catalog.yml -v

  To build a sfincs model based on a bbox hydromt build sfincs
  /path/to/model_root  -i /path/to/sfincs_config.yml  -r "{'bbox':
  [4.6891,52.9750,4.9576,53.1994]}"  -d /path/to/data_catalog.yml -v

Options:
  --opt TEXT               Method specific keyword arguments, see the method
                           documentation of the specific model for more
                           information about the arguments.
  -i, --config PATH        Path to hydroMT configuration file, for the model
                           specific implementation.
  -r, --region TEXT        Set the region for which to 

### Model setup configuration

The HydroMT configuration file (YAML) contains the model setup configuration and determines which methods are used to prepare the different components of a Wflow Sediment model and in which order and optionally sets non-default arguments for each method. This configuration is passed to HydroMT using `-i <path_to_config_file>`. We have prepared several example files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_wflow/latest/user_guide/sediment_build.html). 

Each header (without indent) (e.g. `setup_basemaps:`) corresponds to a model method. All model methods are explained in the [docs (model methods)](https://deltares.github.io/hydromt_wflow/latest/user_guide/sediment_model_setup.html). 

We will load the default wflow sediment build configuration yaml file for inspection:

In [3]:
fn_config = "wflow_sediment_build.yml"
with open(fn_config, "r") as f:
    txt = f.read()
print(txt)

setup_config:                     # options parsed to wflow toml file <section>.<option>
  starttime: 2010-01-01T00:00:00
  endtime: 2010-03-31T00:00:00
  timestepsecs: 86400

setup_basemaps:
  hydrography_fn: merit_hydro      # source hydrography data {merit_hydro, merit_hydro_1k}
  basin_index_fn: merit_hydro_index # source of basin index corresponding to hydrography_fn
  upscale_method: ihu              # upscaling method for flow direction data, by default 'ihu'
  res: 0.00833           # build the model at a 30 arc sec (~1km) resolution

setup_rivers:
  hydrography_fn: merit_hydro      # source hydrography data, should correspond to hydrography_fn in setup_basemaps
  river_geom_fn: hydro_rivers_lin # river source data with river width and bankfull discharge
  river_upa: 30               # minimum upstream area threshold for the river map [km2]
  rivdph_method: powlaw           # method to estimate depth {'powlaw', 'manning', 'gvf'}
  min_rivdph: 1                # minimum river de

### Setup Wflow Sediment model from scratch

In [4]:
# NOTE: copy this line (without !) to your shell for more direct feedback
# NOTE: as of hydromt version 0.7.0 the region argument is optional and should be preceded by -r or --region
# NOTE: the resolution argument has been removed since hydromt v0.7.0 and can now only be configured from the configuration file under the setup_basemap header
!hydromt build wflow_sediment "./wflow_test_sediment" -r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -i wflow_sediment_build.yml -d artifact_data -vv

2025-04-09 16:14:11,707 - build - log - DEBUG - Writing log messages to new file C:\code\py_dev\hydromt_wflow\docs\_examples\wflow_test_sediment\hydromt.log.
2025-04-09 16:14:11,707 - build - log - INFO - HydroMT version: 0.10.1
2025-04-09 16:14:11,707 - build - main - INFO - Building instance of wflow_sediment model at C:\code\py_dev\hydromt_wflow\docs\_examples\wflow_test_sediment.
2025-04-09 16:14:11,707 - build - main - INFO - User settings:
2025-04-09 16:14:12,131 - build - data_catalog - INFO - Reading data catalog artifact_data latest
2025-04-09 16:14:12,131 - build - data_catalog - INFO - Parsing data catalog from C:\Users\dalmijn\.hydromt_data\artifact_data\v0.0.9\data_catalog.yml
2025-04-09 16:14:12,743 - build - model_api - INFO - Initializing wflow_sediment model from hydromt_wflow (v0.8.0).
2025-04-09 16:14:12,748 - build - data_catalog - INFO - Parsing data catalog from C:\code\py_dev\hydromt_wflow\hydromt_wflow\data\parameters_data.yml
2025-04-09 16:14:12,758 - build - m

The example above means the following: run **hydromt build** with:

* `wflow_sediment` : i.e. build a wflow sediment model
* `./wflow_test_sediment` : output model folder
* `-r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/model_region)
* `-i wflow_sediment_build.yml` : model setup configuration file
* `-d artifact_data` : use pre-defined artifact_data catalog.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow run configuration (.toml) file. These files are sufficient to run the wflow sediment model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [5]:
import os

root = "wflow_test_sediment"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith(".xml"):
            continue
        print(f" - {name}")

wflow_test_sediment
 - hydromt.log
 - hydromt_data.yml
 - staticmaps.nc
 - wflow_sediment.toml
wflow_test_sediment\instate
wflow_test_sediment\run_default
wflow_test_sediment\staticgeoms
 - basins.geojson
 - basins_highres.geojson
 - gauges.geojson
 - gauges_grdc.geojson
 - lakes.geojson
 - region.geojson
 - rivers.geojson


### Visualize and/or inspect model schematization

You can (copy and) adapt the following example notebooks in order to visualize your Wflow Sediment model: 

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.

### Add Wflow Sediment layers to an existing Wflow model

If you already have a hydrological **Wflow** model and you want to extend it in order to include **sediment** as well, then you do not need to build the wflow_sediment from scratch. You can instead **update** the wflow model with the additional components needed by wflow_sediment. 

These components are available in the template **wflow_extend_sediment.yml**:

In [6]:
fn_config = "wflow_extend_sediment.yml"
with open(fn_config, "r") as f:
    txt = f.read()
print(txt)

setup_config: # options parsed to wflow toml file <section>.<option>
  starttime: 2010-02-02T00:00:00
  endtime: 2010-02-10T00:00:00
  timestepsecs: 86400
  input.path_forcing: inmaps-era5-2010.nc
#   model.dolake: True               # uncomment if there are lakes in the wflow model
#   model.doreservoir: True          # uncomment if there are reservoirs in the wflow model

setup_riverbedsed:
  bedsed_mapping_fn:       # path to a mapping csv file from streamorder to river bed particles characteristics if any, else default is used

setup_lulcmaps:
  lulc_fn: globcover_2009        # source for lulc maps: {globcover, vito, corine}
  lulc_mapping_fn: globcover_mapping_default  # default mapping for lulc classes

setup_canopymaps:
  canopy_fn: simard           # source for vegetation canopy height: {simard}

setup_soilmaps:
  soil_fn: soilgrids        # source for soilmaps: {soilgrids}
  usleK_method: renard           # method to compute the USLE K factor: {renard, epic}

setup_gauges:
  g

Let's update the hydrological **Wflow** model available from our example for the Piave subbasin:

In [7]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt update wflow_sediment "wflow_piave_subbasin" -o "./wflow_test_extend_sediment" -i wflow_extend_sediment.yml -d artifact_data -vv

2025-04-09 16:14:46,305 - update - log - DEBUG - Writing log messages to new file C:\code\py_dev\hydromt_wflow\docs\_examples\wflow_test_extend_sediment\hydromt.log.
2025-04-09 16:14:46,305 - update - log - INFO - HydroMT version: 0.10.1
2025-04-09 16:14:46,305 - update - main - INFO - Updating wflow_sediment model at C:\code\py_dev\hydromt_wflow\docs\_examples\wflow_piave_subbasin (r).
2025-04-09 16:14:46,305 - update - main - INFO - Output dir: C:\code\py_dev\hydromt_wflow\docs\_examples\wflow_test_extend_sediment
2025-04-09 16:14:46,305 - update - main - INFO - User settings:
2025-04-09 16:14:46,777 - update - data_catalog - INFO - Reading data catalog artifact_data latest
2025-04-09 16:14:46,779 - update - data_catalog - INFO - Parsing data catalog from C:\Users\dalmijn\.hydromt_data\artifact_data\v0.0.9\data_catalog.yml
2025-04-09 16:14:47,378 - update - log - DEBUG - Appending log messages to file C:\code\py_dev\hydromt_wflow\docs\_examples\wflow_piave_subbasin\hydromt.log.
2025-

The example above means the following: run **hydromt update** with:

* `wflow_sediment` : i.e. update a wflow_sediment model (in our case a wflow model but with wflow_sediment components)
* `wflow_piave_subbasin` : hydrological wflow model folder
* `-o "./wflow_test_extend_sediment"`: output combined wflow hydrology+sediment models
* `-i wflow_extend_sediment.yml` : setup configuration file containing wflow sediment specific components
* `-d artifact_data` : use the pre-defined artifact_data catalog
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.